In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
# Cargar los datos de entrenamiento desde un archivo CSV
data = pd.read_csv("card_transdata.csv")

In [3]:
data.head()

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
0,57.877857,0.311140,1.945940,1.0,1.0,0.0,0.0,0.0
1,10.829943,0.175592,1.294219,1.0,0.0,0.0,0.0,0.0
2,5.091079,0.805153,0.427715,1.0,0.0,0.0,1.0,0.0
3,2.247564,5.600044,0.362663,1.0,1.0,0.0,1.0,0.0
4,44.190936,0.566486,2.222767,1.0,1.0,0.0,1.0,0.0


In [4]:
data.rename(columns={
    'distance_from_home': 'distancia_de_casa',
    'distance_from_last_transaction': 'distancia_desde_ultima_transaccion',
    'ratio_to_median_purchase_price': 'relacion_con_precio_medio_de_compra',
    'repeat_retailer': 'minorista_habitual',
    'used_chip': 'chip_usado',
    'used_pin_number': 'numero_pin_usado',
    'online_order': 'comprar_online',
    'fraud': 'fraude'
}, inplace=True)

In [5]:
X = data.drop("fraude", axis=1)  # Características de entrenamiento
y = data["fraude"]  # Etiquetas de entrenamiento

In [6]:
#80% de los datos seran para test y 20% para validación
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size = 0.2,
                                                   random_state = 1,
                                                   stratify = y)

In [7]:
import numpy as np
print('Numero de muestras en y:', np.bincount(y))
print('Numero de muestras en y_train:', np.bincount(y_train))
print('Numero de muestras en y_test:', np.bincount(y_test))

Numero de muestras en y: [912597  87403]
Numero de muestras en y_train: [730078  69922]
Numero de muestras en y_test: [182519  17481]


In [8]:
# Codigo para saber los hiperparemetros de mayor rendimiento 
# su ejecucion demora un tiempo considerable dependiendo de los recursos de hardhware
"""
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

n_estimators_range = range(3, 8)
max_depth_range = range(3, 13, 3)
param_grid = {'n_estimators': n_estimators_range, 'max_depth': max_depth_range, 'random_state': [5]}

start_time = time.time()

rfc_grid = GridSearchCV(estimator=RandomForestClassifier(), param_grid=param_grid, scoring='accuracy')
rfc_grid.fit(X_train, y_train)

elapsed_time = time.time() - start_time

print('RFC tuning time: ', elapsed_time)
print('Best score obtained with RFC: ' + str(rfc_grid.best_score_))
print('by setting', rfc_grid.best_params_)

results = rfc_grid.cv_results_
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

x = results['param_max_depth']
y = results['param_n_estimators']
z = results['mean_test_score']
ax.scatter(x, y, z)
ax.set_xlabel('max_depth')
ax.set_ylabel('n_estimators')
ax.set_zlabel('mean_test_score')
"""

"\nimport time\nimport matplotlib.pyplot as plt\nfrom sklearn.model_selection import GridSearchCV\n\nn_estimators_range = range(3, 8)\nmax_depth_range = range(3, 13, 3)\nparam_grid = {'n_estimators': n_estimators_range, 'max_depth': max_depth_range, 'random_state': [5]}\n\nstart_time = time.time()\n\nrfc_grid = GridSearchCV(estimator=RandomForestClassifier(), param_grid=param_grid, scoring='accuracy')\nrfc_grid.fit(X_train, y_train)\n\nelapsed_time = time.time() - start_time\n\nprint('RFC tuning time: ', elapsed_time)\nprint('Best score obtained with RFC: ' + str(rfc_grid.best_score_))\nprint('by setting', rfc_grid.best_params_)\n\nresults = rfc_grid.cv_results_\nfig = plt.figure()\nax = fig.add_subplot(projection='3d')\n\nx = results['param_max_depth']\ny = results['param_n_estimators']\nz = results['mean_test_score']\nax.scatter(x, y, z)\nax.set_xlabel('max_depth')\nax.set_ylabel('n_estimators')\nax.set_zlabel('mean_test_score')\n"

In [9]:
# Ajustes del modelo
n_estimators = 7
criterion = 'gini'
max_depth = 9
random_state = 5
# Crear instancia del modelo
model = RandomForestClassifier(n_estimators=n_estimators,
                                criterion=criterion,
                                max_depth=max_depth,
                                random_state=random_state
                                )

In [10]:
import time
start_time = time.time()
model.fit(X_train, y_train)  #entranamos el modelo
elapsed_time = time.time() - start_time
print('RFC tuning time: ', elapsed_time)

RFC tuning time:  6.934939622879028


In [11]:
"""
#metricas para saber la presicion del modelo
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
y_rfc = model.predict(X_test)
cm = confusion_matrix(y_true=y_test, y_pred=y_rfc)
ConfusionMatrixDisplay(cm).plot(cmap='Blues', colorbar=False)
plt.title('RFC Confusion Matrix')
plt.show()

report = pd.DataFrame(data=classification_report(
    y_true=y_test, y_pred=y_rfc, output_dict=True))
report
"""

"\n#metricas para saber la presicion del modelo\nimport matplotlib.pyplot as plt\nfrom sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report\ny_rfc = model.predict(X_test)\ncm = confusion_matrix(y_true=y_test, y_pred=y_rfc)\nConfusionMatrixDisplay(cm).plot(cmap='Blues', colorbar=False)\nplt.title('RFC Confusion Matrix')\nplt.show()\n\nreport = pd.DataFrame(data=classification_report(\n    y_true=y_test, y_pred=y_rfc, output_dict=True))\nreport\n"

In [12]:
# Realizar predicciones en el conjunto de prueba
"""
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
y_pred = model.predict(X_test)

# Evaluar el rendimiento del modelo
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
"""

'\nfrom sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score\ny_pred = model.predict(X_test)\n\n# Evaluar el rendimiento del modelo\naccuracy = accuracy_score(y_test, y_pred)\nprecision = precision_score(y_test, y_pred)\nrecall = recall_score(y_test, y_pred)\nf1 = f1_score(y_test, y_pred)\n\nprint("Accuracy:", accuracy)\nprint("Precision:", precision)\nprint("Recall:", recall)\nprint("F1-score:", f1)\n'

In [13]:
# Precision global de clasificación correcta
"""
print('Train Accuracy : %.5f' % model.score(X_train, y_train))
print('Test Accuracy : %.5f' % model.score(X_test, y_test))
"""

"\nprint('Train Accuracy : %.5f' % model.score(X_train, y_train))\nprint('Test Accuracy : %.5f' % model.score(X_test, y_test))\n"

In [19]:
"""
distance_from_home - la distancia desde casa donde ocurrió la transacción.

distance_from_last_transaction - distancia desde la última transacción realizada.

ratio_to_median_purchase_price - Relación entre el precio de compra de la transacción y el precio medio de compra.

repeat_retailer - La transacción se ha realizado en el mismo minorista.

used_chip - La transacción se ha realizado mediante chip (tarjeta de crédito).

used_pin_number - La transacción se ha realizado utilizando un número PIN.

online_order - La transacción es un pedido en línea.

fraud - La transacción es fraudulenta.
"""

'\ndistance_from_home - la distancia desde casa donde ocurrió la transacción.\n\ndistance_from_last_transaction - distancia desde la última transacción realizada.\n\nratio_to_median_purchase_price - Relación entre el precio de compra de la transacción y el precio medio de compra.\n\nrepeat_retailer - La transacción se ha realizado en el mismo minorista.\n\nused_chip - La transacción se ha realizado mediante chip (tarjeta de crédito).\n\nused_pin_number - La transacción se ha realizado utilizando un número PIN.\n\nonline_order - La transacción es un pedido en línea.\n\nfraud - La transacción es fraudulenta.\n'

In [14]:
# Para usar este codigo debe ver el siguiente video, la contraseña no esta incluida por seguridad
# https://www.youtube.com/watch?v=DDVpKvJXRz8
import ssl
import smtplib
from email.message import EmailMessage
with open('contrasena.txt', 'r') as file:
    email_contrasena = file.readline().strip()

In [17]:
# Variables para almacenar las transacciones simuladas y las alertas generadas
import datetime
simulated_transactions = []
alerts = []
tarjetas_bloqueadas = []
tarjetas_registradas = {}
tarjetas_transacciones = {}


def enviar_correo(email_receptor):
    email_emisor = 'esteocalu@gmail.com'
    asunto = 'ALERTA DE FRAUDE'
    cuerpo = """SE DETECTO UNA TRANSACCION SOSPECHOSA Y SE BLOQUEO SU TARJETA.
POR FAVOR LLAMAR A ATENCIÓN AL CLIENTE PARA DESBLOQUEAR TARJETA"""
    em = EmailMessage()
    em['From'] = email_emisor
    em['To'] = email_receptor
    em['Subject'] = asunto
    em.set_content(cuerpo)

    contexto = ssl.create_default_context()

    with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=contexto) as smtp:
        smtp.login(email_emisor, email_contrasena)
        smtp.sendmail(email_emisor, email_receptor, em.as_string())
        
def validate_card_number(numero_tarjeta):
    if len(numero_tarjeta) != 16:
        print("El número de tarjeta debe contener exactamente 16 dígitos.")
        return False
    return True

def simulate_transaction(numero_tarjeta):
    global simulated_transactions, alerts, tarjetas_bloqueadas
    if numero_tarjeta in tarjetas_bloqueadas:
        print("La tarjeta está bloqueada. No se puede realizar la transacción.")
        return False
    else:
        distancia_de_casa = float(input("Ingrese la distancia desde casa: "))
        distancia_desde_ultima_transaccion = float(input("Ingrese la distancia desde la última transacción: "))
        relacion_con_precio_medio_de_compra = float(input("Ingrese la relación con el precio medio de compra: "))
        minorista_habitual = int(input("¿Es un minorista habitual? (1 para Sí, 0 para No): "))
        chip_usado = int(input("¿Se utilizó el chip de la tarjeta? (1 para Sí, 0 para No): "))
        numero_pin_usado = int(input("¿Se utilizó el número PIN de la tarjeta? (1 para Sí, 0 para No): "))
        comprar_online = int(input("¿Se realizó la compra en línea? (1 para Sí, 0 para No): "))

        # Realizar la predicción del modelo para determinar si la transacción es fraudulenta o no
        prediction = model.predict([[distancia_de_casa, distancia_desde_ultima_transaccion,
                                     relacion_con_precio_medio_de_compra, minorista_habitual,
                                     chip_usado, numero_pin_usado, comprar_online]])[0]

        if prediction == 1:
            # print("¡Alerta de fraude! La transacción ha sido marcada como fraudulenta.")
            alerts.append((distancia_de_casa, distancia_desde_ultima_transaccion,
                           relacion_con_precio_medio_de_compra, minorista_habitual,
                           chip_usado, numero_pin_usado, comprar_online, numero_tarjeta))
            email_receptor= input("""¡Alerta de fraude! La transacción ha sido marcada como fraudulenta.
Ingrese el correo electronico del cliente:""" )
            #enviar_correo(email_receptor) para usar esta funcion se debe revisar la celda anterior
            print("El correo electrónico de alerta ha sido enviado.")
            
        else:
            print("La transacción es legítima.")

        simulated_transactions.append((distancia_de_casa, distancia_desde_ultima_transaccion,
                                       relacion_con_precio_medio_de_compra, minorista_habitual,
                                       chip_usado, numero_pin_usado, comprar_online))
    return True

def manage_card():
    global tarjetas_bloqueadas, tarjetas_registradas, tarjetas_transacciones
    print("\n----- Gestión de tarjeta -----")
    print("a. Bloqueo de tarjeta")
    print("b. Desbloqueo de tarjeta")
    print("c. Registro de tarjeta")
    sub_option = input("Ingrese una opción: ")

    if sub_option == "a":
        print("\n----- Bloqueo de tarjeta -----")
        card_to_block = input("Ingrese el número de tarjeta a bloquear: ")
        tarjetas_bloqueadas.append(card_to_block)
        tarjetas_registradas[card_to_block] = "Bloqueada"
        tarjetas_transacciones.setdefault(card_to_block, []).append(("Bloqueo", datetime.datetime.now()))
        print("La tarjeta ha sido bloqueada.")

    elif sub_option == "b":
        print("\n----- Desbloqueo de tarjeta -----")
        card_to_unblock = input("Ingrese el número de tarjeta a desbloquear: ")
        if card_to_unblock in tarjetas_bloqueadas:
            tarjetas_bloqueadas.remove(card_to_unblock)
            tarjetas_registradas[card_to_unblock] = "Desbloqueada"
            tarjetas_transacciones.setdefault(card_to_unblock, []).append(("Desbloqueo", datetime.datetime.now()))
            print("La tarjeta ha sido desbloqueada.")
        else:
            print("La tarjeta no está bloqueada.")

    elif sub_option == "c":
        print("\n----- Registro de tarjeta -----")
        print("Estado actual de las tarjetas:")
        for tarjeta, estado in tarjetas_registradas.items():
            print(f"Número de tarjeta: {tarjeta}, Estado: {estado}")
            if tarjeta in tarjetas_transacciones:
                print("Transacciones:")
                for transaccion in tarjetas_transacciones[tarjeta]:
                    print(f"Tipo: {transaccion[0]}, Fecha y hora: {transaccion[1]}")
            else:
                print("No hay transacciones registradas para esta tarjeta.")
    else:
        print("Opción inválida. Por favor, seleccione una opción válida.")

def display_alerts():
    global alerts
    print("\n----- Alertas generadas -----")
    for alert in alerts:
        print(f"Número de tarjeta: {alert[7]}")
        print(f"Distancia de casa: {alert[0]}, Distancia desde última transacción: {alert[1]}, "
              f"Relación con precio medio de compra: {alert[2]}, Minorista habitual: {alert[3]}, "
              f"Chip usado: {alert[4]}, Número PIN usado: {alert[5]}, Comprar en línea: {alert[6]}")

def main_menu():
    global simulated_transactions
    while True:
        print("\n----- Menú Principal -----")
        print("1. Simulación de transacción")
        print("2. Alertas generadas")
        print("3. Gestión de tarjeta")
        print("4. Salir")

        option = input("Ingrese una opción: ")

        if option == "1":
            print("\n----- Simulación de transacción -----")
            numero_tarjeta = input("Ingrese el número de tarjeta de crédito: ")
            if validate_card_number(numero_tarjeta):
                simulate_transaction(numero_tarjeta)

        elif option == "2":
            display_alerts()

        elif option == "3":
            manage_card()

        elif option == "4":
            break

        else:
            print("Opción inválida. Por favor, seleccione una opción válida.")

    print("¡Gracias por usar el programa!")

In [ ]:
main_menu()


----- Menú Principal -----
1. Simulación de transacción
2. Alertas generadas
3. Gestión de tarjeta
4. Salir
Ingrese una opción: 1

----- Simulación de transacción -----
Ingrese el número de tarjeta de crédito: 1234567812345678
La tarjeta está bloqueada. No se puede realizar la transacción.

----- Menú Principal -----
1. Simulación de transacción
2. Alertas generadas
3. Gestión de tarjeta
4. Salir
Ingrese una opción: 2

----- Alertas generadas -----
Número de tarjeta: 1234567812345678
Distancia de casa: 22.6, Distancia desde última transacción: 0.01, Relación con precio medio de compra: 4.6, Minorista habitual: 1, Chip usado: 0, Número PIN usado: 0, Comprar en línea: 1

----- Menú Principal -----
1. Simulación de transacción
2. Alertas generadas
3. Gestión de tarjeta
4. Salir
Ingrese una opción: 2

----- Alertas generadas -----
Número de tarjeta: 1234567812345678
Distancia de casa: 22.6, Distancia desde última transacción: 0.01, Relación con precio medio de compra: 4.6, Minorista habit